In [1]:
import sys
sys.path.append('../RecSysRep/')

In [2]:
import Basics.Load as ld

URM_all, _, _, _, _ = ld.getCOOs()
ICM_all = ld.getICMall()

In [3]:
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8, seed=8888)
# URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.8)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
# evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 13643 ( 0.1%) Users that have less than 1 test interactions


In [4]:
import os

ofp = "../models_temp/Similarity_Hybrid/"

models_to_combine_best = {
                            'RP3ICMnew': {'alpha': 1.029719677583138, 'beta': 1.0630164752134375, 'topK': 6964, 'normalize_similarity': True},
                            'RP3ICM' : {"topK": 2550, "alpha": 1.3058102610510849, "beta": 0.5150718337969987, "normalize_similarity": True, "implicit": True},
                            'IALS' : {"num_factors": 29, "epochs": 50, "confidence_scaling": "log", "alpha": 0.001, "epsilon": 0.001, "reg": 0.01},
                            'SLIMgensub': {"l1_ratio" : 0.025887359156206147, "topK": 2140, "alpha": 0.009567288586539689, "workers": 8, "mw": 1},
                            'SLIMBPR' : {"epochs": 440, "lambda_i": 0.007773815998802306, "lambda_j": 0.003342522366982381, "learning_rate": 0.010055161410725193, "topK": 4289, "random_seed": 1234, "sgd_mode": "sgd"},
                            'SLIMweig': {'l1_ratio': 0.0005247075138160404, 'topK': 4983, 'alpha': 0.06067400905430761, 'workers': 8, 'mw': 2.308619939318322},
                            'SLIMER': {'topK': 6000, 'l1_ratio': 0.0005495104968035837, 'alpha': 0.08007142704041009, 'workers': 8},
                            'P3alpha': {'topK': 4834, 'alpha': 1.764994849187595, 'normalize_similarity': True, 'implicit': True},
                            'RP3beta': {"topK": 1049, "alpha": 1.1626473723475605, "beta": 0.6765017195261293, "normalize_similarity": True, "implicit": True},
                            'sel7': {"shrink": 4000, "topK": 985, "feature_weighting": "TF-IDF", "normalize": True},
                            'sel9': {"shrink": 5212, "topK": 923, "feature_weighting": "TF-IDF", "normalize": True},
                            'sel3': {'shrink': 2211, 'topK': 188, 'feature_weighting': 'TF-IDF', 'normalize': True},
                            'ICM_all': {"shrink": 5675, "topK": 2310, "feature_weighting": "BM25", "normalize": False},
                            'ER_BPR': {'alpha': 0.58},
                            'TopPop': {},
                         }


In [5]:
def model_init(recommender, name, args):
    path_name = ofp + name + ".zip"
    print(path_name)
    if os.path.exists(path_name):
        print("Model found!")
        recommender.load_model(ofp, name)
    else:
        print("Model does not exists, creating...")
        if not os.path.exists(ofp):
            print("Main folder does not exist, creating...")
            os.makedirs(ofp)
        recommender.fit(**args)
        recommender.save_model(ofp, name)

In [6]:
from Recommenders.NonPersonalizedRecommender import TopPop
from Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommenderICM
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender

In [7]:
from Recommenders.KNN.ItemKNNScoresHybridMultipleRecommender import ItemKNNScoresHybridTwoRecommender
from Recommenders.KNN.ItemKNNScoresHybridMultipleRecommender import ItemKNNScoresHybridMultipleRecommender
from Recommenders.KNN.ItemKNNScoresHybridMultipleRecommender import ItemKNNScoresHybridTwoRecommender_PRUNE

In [8]:
recommender1 = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train)
recommender2 = SLIM_BPR_Cython(URM_train)
recommender3 = RP3betaRecommender(URM_train)
# recommender3 = ItemKNNCBFRecommender(URM_train, ICM_sel_9)

model_init(recommender1, 'SLIMER', models_to_combine_best['SLIMER'])
# model_init(recommender2, 'KNNweigh', models_to_combine_best['icm_weighted'])
model_init(recommender2, 'SLIMBPR_8888', models_to_combine_best['SLIMBPR'])
model_init(recommender3, 'RP3beta', models_to_combine_best['RP3beta'])
# model_init(recommender3, 'TopPop', models_to_combine_best['TopPop'])


../models_temp/Similarity_Hybrid/SLIMER.zip
Model found!
SLIMElasticNetRecommender: Loading model from file '../models_temp/Similarity_Hybrid/SLIMER'
SLIMElasticNetRecommender: Loading complete
../models_temp/Similarity_Hybrid/SLIMBPR_8888.zip
Model found!
SLIM_BPR_Recommender: Loading model from file '../models_temp/Similarity_Hybrid/SLIMBPR_8888'
SLIM_BPR_Recommender: Loading complete
../models_temp/Similarity_Hybrid/RP3beta.zip
Model found!
RP3betaRecommender: Loading model from file '../models_temp/Similarity_Hybrid/RP3beta'
RP3betaRecommender: Loading complete


In [ ]:
recommenderHybrid = ItemKNNScoresHybridTwoRecommender(URM_train, recommender1, recommender2)
recommenderHybrid.fit(alpha = 0.58)


In [ ]:
def rank_models(evaluator, recommenders):
    for r in recommenders:
        r_d, _ = evaluator.evaluateRecommender(r)
        print(r.RECOMMENDER_NAME, r_d.loc[10]['MAP'])

In [ ]:
rank_models(evaluator_validation, [recommender1, recommender2])

In [9]:
# !pip install optuna
import optuna

class Objective(object):
    def __init__(self, URM_train, recommender1, recommender2, recommender3, evaluator):
        # Hold this implementation specific arguments as the fields of the class.
        self.URM_train = URM_train
        self.evaluator = evaluator
        self.recommender_1 = recommender1
        self.recommender_2 = recommender2
        self.recommender_3 = recommender3

    def __call__(self, trial):
        # Calculate an objective value by using the extra arguments.
        alpha = trial.suggest_uniform('alpha', 0, 1)
        beta = trial.suggest_uniform('beta', 0, 1)
        gamma = trial.suggest_uniform('gamma', 0, 1)

        recommender_final = ItemKNNScoresHybridMultipleRecommender(self.URM_train.tocsr(), self.recommender_1, self.recommender_2, self.recommender_3)
        recommender_final.fit(alpha, beta, gamma)
        result_dict, _ = self.evaluator.evaluateRecommender(recommender_final)

        map_v = -result_dict.loc[10]['MAP']
        print(map_v)
        if map_v < -0.245: # minimum acceptable map
            return map_v
        else:
            # Calculate the penalty.
            penalty = 0.245 + map_v
            trial.report(penalty, 0)

            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()
            return map_v

     |████████████████████████████████| 308 kB 2.1 MB/s eta 0:00:01
     |████████████████████████████████| 80 kB 3.7 MB/s  eta 0:00:01
     |████████████████████████████████| 209 kB 48.9 MB/s eta 0:00:01
     |████████████████████████████████| 75 kB 4.1 MB/s  eta 0:00:01
     |████████████████████████████████| 149 kB 44.2 MB/s eta 0:00:01
     |████████████████████████████████| 49 kB 2.5 MB/s  eta 0:00:01
     |████████████████████████████████| 112 kB 6.2 MB/s eta 0:00:01
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=2004f5c9dfa2b34048a6003714057c9c89cb84fd911e990e3b75e6c0b5253ab4
  Stored in directory: /home/alessio/.cache/pip/wheels/7f/1a/65/84ff8c386bec21fca6d220ea1f5498a0367883a78dd5ba6122
Successfully built pyperclip


In [10]:
study = optuna.create_study(direction='minimize')
study.optimize(Objective(URM_train, recommender1, recommender2, recommender3, evaluator_validation), n_trials=500)

print(study.best_params)

[I 2022-01-04 13:06:16,685] A new study created in memory with name: no-name-25a73f4a-d679-446d-a588-34a60e9329bb


CURRENT CONFIGURATION:
SLIMElasticNetRecommender with weight alpha: 0.28667858981765215
SLIM_BPR_Recommender with weight beta: 0.47213680406476677
RP3betaRecommender with weight gamma: 0.24118460611758114
EvaluatorHoldout: Processed 13643 (100.0%) in 3.89 min. Users per second: 58


[I 2022-01-04 13:10:10,032] Trial 0 finished with value: -0.23130503287341897 and parameters: {'alpha': 0.576284879991134, 'beta': 0.9490952974302237, 'gamma': 0.4848323060350872}. Best is trial 0 with value: -0.23130503287341897.


-0.23130503287341897
CURRENT CONFIGURATION:
SLIMElasticNetRecommender with weight alpha: 0.22361077618735503
SLIM_BPR_Recommender with weight beta: 0.49103217970646107
RP3betaRecommender with weight gamma: 0.2853570441061839
EvaluatorHoldout: Processed 13643 (100.0%) in 3.80 min. Users per second: 60


[I 2022-01-04 13:13:58,143] Trial 1 finished with value: -0.23080460763063432 and parameters: {'alpha': 0.44522034698650526, 'beta': 0.9776698652809077, 'gamma': 0.5681602843932306}. Best is trial 0 with value: -0.23130503287341897.


...

[I 2022-01-05 07:38:52,338] Trial 289 finished with value: -0.2468509026665219 and parameters: {'alpha': 0.9324581743974305, 'beta': 0.0646449021994032, 'gamma': 0.027066479210246793}. Best is trial 173 with value: -0.25019763609223405.


-0.2468509026665219
CURRENT CONFIGURATION:
SLIMElasticNetRecommender with weight alpha: 0.9195050527795717
SLIM_BPR_Recommender with weight beta: 0.0006047938989585346
RP3betaRecommender with weight gamma: 0.07989015332146977
EvaluatorHoldout: Processed 13643 (100.0%) in 3.82 min. Users per second: 60


[I 2022-01-05 07:42:41,542] Trial 290 finished with value: -0.24908259250339423 and parameters: {'alpha': 0.9575183856387043, 'beta': 0.0006297967325185963, 'gamma': 0.08319289862036286}. Best is trial 173 with value: -0.25019763609223405.


-0.24908259250339423
CURRENT CONFIGURATION:
SLIMElasticNetRecommender with weight alpha: 0.8598506724860122
SLIM_BPR_Recommender with weight beta: 0.028080696953612885
RP3betaRecommender with weight gamma: 0.11206863056037478


KeyboardInterrupt: 

In [ ]:
import numpy as np

# TWO RECOMMENDERS

def test_percentage(recommender_1, recommender_2, evaluator, step):
    recommender = ItemKNNScoresHybridTwoRecommender_PRUNE(URM_train, recommender_1, recommender_2)
    results = []
    alp_space = np.linspace(0.5, 0.55, step, True)
    for alp in alp_space:
        recommender.fit(alpha = alp)
        r_d, _ = evaluator.evaluateRecommender(recommender)
        print(alp, ":", r_d.loc[10]['MAP'])
        results.append(r_d.loc[10]['MAP'])
    
    return alp_space, results

In [ ]:
alp_space, results = test_percentage(recommender1, recommender2, evaluator_validation, 2)

In [ ]:
import matplotlib.pyplot as plt

_ = plt.figure(figsize=(16, 9))
plt.plot(alp_space,results, label='MAP variability')
plt.ylabel('MAP')
plt.xlabel('User Group')
plt.legend()
plt.show()

plt.savefig('userwise.png')

In [ ]:

recommender1 = MultiThreadSLIM_SLIMElasticNetRecommender(URM_all)
# recommender1.fit(**models_to_combine_best['SLIMER'])
# recommender1.save_model(ofp + 'SLIMER_ALL/', 'SLIMER_ALL')
recommender1.load_model(ofp + 'SLIMER_ALL/', 'SLIMER_ALL')

recommender2 = SLIM_BPR_Cython(URM_all)
# recommender2.fit(**models_to_combine_best['SLIMBPR'])
# recommender2.save_model(ofp + 'SLIMBPR_ALL/', 'SLIMBPR_ALL')
recommender2.load_model(ofp + 'SLIMBPR_ALL/', 'SLIMBPR_ALL')

recommenderHybrid = ItemKNNScoresHybridTwoRecommender(URM_all, recommender1, recommender2)
recommenderHybrid.fit(alpha = 0.58)

recommender = recommenderHybrid

import pandas as pd
at = 10

user_test_path = '../data/data_target_users_test.csv'
user_test_dataframe = pd.read_csv(filepath_or_buffer=user_test_path,
sep=",",
dtype={0:int})

subm_set = user_test_dataframe.to_numpy().T[0]


subm_res = {"user_id":[], "item_list":[]}

for user_id in subm_set:
	subm_res["user_id"].append(user_id)
	res = recommender.recommend(user_id, cutoff=at)
	res = ' '.join(map(str, res))
	if user_id < 3:
		print(user_id)
		print(res)
	subm_res["item_list"].append(res)


	# print(subm_res)

submission = pd.DataFrame.from_dict(subm_res)
	# submission

from datetime import datetime

now = datetime.now() # current date and time


submission.to_csv('../subs/submission {:%Y_%m_%d %H_%M_%S}.csv'.format(now), index=False)



In [ ]:
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
CFrecommender = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train)
CFrecommender.load_model('../result_experiments/BEST_SLIM_PURECF/', file_name = CFrecommender.RECOMMENDER_NAME + "_my_own_save.zip")

from Recommenders.FeatureWeighting.CFW_D_Similarity_Linalg import CFW_D_Similarity_Linalg
FW_recommender = CFW_D_Similarity_Linalg(URM_train, ICM_stacked, CFrecommender.W_sparse)
FW_recommender.load_model('../result_experiments/BEST_FW/', file_name = FW_recommender.RECOMMENDER_NAME + "_my_own_save.zip")

In [ ]:
import scipy.sparse as sps
ICM_length_all_5km = ld.getICMlength('5km')
ICM_stacked = sps.hstack((ICM_genre_all, ICM_subgenre_all, ICM_channel_all, ICM_length_all_5km))

In [ ]:
import pandas as pd
argsort_features = np.argsort(-FW_recommender.D_best)

selection_quota = 0.1
        
n_to_select = int(selection_quota*len(argsort_features))
selected_features = argsort_features[:n_to_select]
        
ICM_selected = ICM_stacked.todense()[:,selected_features]
pd.DataFrame(ICM_selected).to_csv('ICM_selected_1.csv', index=False, header=True)


In [ ]:
import scipy.sparse as sps
import pandas as pd
URM_all, ICM_genre_all, ICM_subgenre_all, ICM_channel_all, _ = ld.getCOOs()
ICM_length_all_5km = ld.getICMlength('5km')
pd.DataFrame(sps.hstack([ICM_genre_all, ICM_subgenre_all, ICM_channel_all, ICM_length_all_5km]).todense()).to_csv('ICM_ALL.csv', index=False, header=True)

In [ ]:
recommender1 = recommender1 = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train)
model_init(recommender1, 'SLIMER', models_to_combine_best['SLIMER'])
rank_models(evaluator_validation, [recommender1])
recommender1.fit(**models_to_combine_best['SLIMER'])
rank_models(evaluator_validation, [recommender1])

# recommender1.save_model(ofp + 'SLIMER_ALL/', 'SLIMER_ALL')
# recommender1.load_model(ofp + 'SLIMER_ALL/', 'SLIMER_ALL')